Signatures
===

* Ultima modificación: Mayo 14, 2022

https://www.mlflow.org/docs/latest/models.html

Código base
---

In [ ]:
def load_data():

    import pandas as pd

    url = "http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
    df = pd.read_csv(url, sep=";")

    y = df["quality"]
    x = df.copy()
    x.pop("quality")

    return x, y

def make_train_test_split(x, y):

    from sklearn.model_selection import train_test_split

    (x_train, x_test, y_train, y_test) = train_test_split(
        x,
        y,
        test_size=0.25,
        random_state=123456,
    )
    return x_train, x_test, y_train, y_test

def eval_metrics(y_true, y_pred):

    from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)

    return mse, mae, r2

def report(estimator, mse, mae, r2):

    print(estimator, ":", sep="")
    print(f"  MSE: {mse}")
    print(f"  MAE: {mae}")
    print(f"  R2: {r2}")
    
def train_estimator(alpha=0.5, l1_ratio=0.5, verbose=1):

    import mlflow.sklearn
    from sklearn.linear_model import ElasticNet

    import mlflow

    x, y = load_data()
    x_train, x_test, y_train, y_test = make_train_test_split(x, y)
    
    print('Tracking directory:', mlflow.get_tracking_uri())
    
    with mlflow.start_run():
    
        estimator = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=12345)
        estimator.fit(x_train, y_train)
        mse, mae, r2 = eval_metrics(y_test, y_pred=estimator.predict(x_test))
        if verbose > 0:
            report(estimator, mse, mae, r2)
         
        mlflow.log_param("alpha", alpha)
        mlflow.log_param("l1_ratio", l1_ratio)
        
        mlflow.log_metric("mse", mse)
        mlflow.log_metric("mae", mae)
        mlflow.log_metric("r2", r2)
        
        mlflow.sklearn.log_model(estimator, "model")
        
        # -------------------------------------------------------------------------------
        #
        # Inferencia de la signatura (tipos de datos del modelo)
        #
        signature = mlflow.models.signature.infer_signature(x_train, estimator.predict(x_train))
        
        mlflow.sklearn.save_model(estimator, "/tmp/my_model", signature=signature)

Almacenamiento del modelo de prueba
---

In [ ]:
!rm -rf mlruns /tmp/my_model
train_estimator(0.1, 0.05)

Signature
---

In [ ]:
#
# Note que en la definición del modelo se agregó signature:
#
!cat /tmp/my_model/MLmodel

Model API
---

Es dependiente de la librería de ML y contiene las siguientes funciones:

* save()

* log()

* load()